<a href="https://colab.research.google.com/github/hardiksharma2005/Reddit-Post-Classifier/blob/main/Reddit_Post_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=7f834fbaac429e901e16d3e282a35b00558a77773521d4ec812ef946e2e210d1
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


In [ ]:
import pandas as pd
import pandasql as ps

Loading the data

In [ ]:
try:
    df = pd.read_csv("reddit data.csv")
    print("✅ Data Loaded Successfully")
    print(df.head())
except:
    print("❌ Error: csv file not found. Please upload 'reddit data.csv'")

✅ Data Loaded Successfully
       id                                              title  score  \
0  ll1p9h  Wordcloud of trending video titles on YouTube ...      1   
1  ll1o4h  Wordcloud of trending videos on YouTube in the...      1   
2  ll15gx  Immunization in India. Source: https://niti.go...      1   
3  ll0iup  How to quickly estimate the impact of players ...      1   
4  ll0g9a  How to quickly estimate the impact of players ...      1   

                 author author_flair_text        removed_by  \
0             OmarZiada             OC: 1               NaN   
1             OmarZiada             OC: 1         moderator   
2  Professional_Napper_               NaN         moderator   
3              Viziball               NaN  automod_filtered   
4              Viziball               NaN         moderator   

   total_awards_received awarders  created_utc  \
0                    0.0       []   1613473961   
1                    0.0       []   1613473829   
2                

/tmp/ipython-input-4151474216.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("reddit data.csv")


Day 1: SQL Analysis using PandaSQL

In [ ]:
if 'over_18' in df.columns:
    df['category'] = df['over_18'].apply(lambda x: 'NSFW' if x == True else 'Safe')
    print("✅ Created 'category' column for SQL analysis.")
else:
    print("⚠️ Column 'over_18' not found. Using generic data.")
    df['category'] = 'Unknown'

✅ Created 'category' column for SQL analysis.


In [ ]:
#Insight: Do NSFW posts get higher scores than Safe posts?
print("\n--- Insight 1: Average Score by Category ---")
q1 = """
SELECT category, COUNT(*) as post_count, AVG(score) as avg_score
FROM df
GROUP BY category
ORDER BY avg_score DESC
"""
print(ps.sqldf(q1, locals()))


--- Insight 1: Average Score by Category ---
  category  post_count   avg_score
0     NSFW        1068  583.908240
1     Safe      189785  173.720779


In [ ]:
# Insight: Classifying posts into 'Viral', 'Hot', or 'Normal' based on score
print("\n--- Insight 2: Viral Post Classification ---")
q2 = """
SELECT title, score,
       CASE
           WHEN score > 5000 THEN 'Viral'
           WHEN score > 1000 THEN 'Hot'
           ELSE 'Normal'
       END as virality_label
FROM df
LIMIT 5
"""
print(ps.sqldf(q2, locals()))


--- Insight 2: Viral Post Classification ---
                                               title  score virality_label
0  Wordcloud of trending video titles on YouTube ...      1         Normal
1  Wordcloud of trending videos on YouTube in the...      1         Normal
2  Immunization in India. Source: https://niti.go...      1         Normal
3  How to quickly estimate the impact of players ...      1         Normal
4  How to quickly estimate the impact of players ...      1         Normal


In [ ]:
# Insight: Find the #1 highest ranked post within each category (Safe vs NSFW)
print("\n--- Insight 3: Top Post per Category (Window Function) ---")
q3 = """
SELECT title, category, score,
       RANK() OVER (PARTITION BY category ORDER BY score DESC) as rank
FROM df
LIMIT 10
"""
try:
    print(ps.sqldf(q3, locals()))
except Exception as e:
    print(f"Window function notice: {e}")


--- Insight 3: Top Post per Category (Window Function) ---
                                               title category  score  rank
0  Pornhub traffic in Hawaii during the "missile ...     NSFW  63569     1
1              Tinder over 3 years (18-21 Male) [OC]     NSFW  59130     2
2  "Fuuuuck", orthographic repetition on Twitter ...     NSFW  38419     3
3  Six months on tinder as a young sex worker loo...     NSFW  34248     4
4  Reddit's favorite swear word is 'shit'; the su...     NSFW  31961     5
5                           Pornhub’s year in review     NSFW  25833     6
6  [oc] How taboo and popular are sexual fetishes...     NSFW  25715     7
7  NSFW Heat Map of Reddit's Favorite Porn Cross-...     NSFW  24508     8
8             On /r/gonewild, 18 year-olds rule [OC]     NSFW  18618     9
9  Playboy’s Cover Girls’ Hair Colors — statistic...     NSFW  17578    10
